In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.base import TransformerMixin
import itertools
from sklearn import preprocessing

<font face="微软雅黑" size=6 color=#000000> Data Processing

In [2]:
data = pd.read_csv('business.csv')
atrribute = pd.read_csv('processed_attributes_round13.csv').drop('Unnamed: 0', axis=1)

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (14,38,39,40,41,42,43,44,76,79) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [84]:
len(data.columns)

99

In [38]:
columns_need = ['attributes','categories', 'city', 'stars', 'state']
data = data[columns_need]

In [52]:
columns_remove = ['stars']
data = data.drop(columns_remove, axis=1)

In [53]:
len(data)

192609

In [21]:
def getCatagories(df):
    
    df = df.dropna().reset_index(drop = True)
    catgories_list = [df[i].split(",") for i in range(df.shape[0])]
    catagories = np.unique(list(itertools.chain.from_iterable(catgories_list)))
    catagories = [tag for tag in catagories if len(tag) > 0]
    
    return catagories
    
    
# one hot catagories
catagories = getCatagories(data["categories"])
data["categories"] = data["categories"].fillna("") 

for tag in catagories:
    col_name = "categories_" + tag
    data[col_name] = data["categories"].map(lambda x: 1 if tag in x.split(",") else 0)
    
data = data.drop(['categories'], axis=1)

In [54]:
data = data.drop(['attributes'], axis=1)

In [23]:
result = pd.concat([data, atrribute], axis=1)

In [34]:
def onehot_encoding(dataset,columns):
    dataset = pd.get_dummies(dataset, columns=columns)
    return dataset

In [35]:
#columns_other = ['city', 'state']
columns_to_onehot = list(atrribute.columns).append(['city','state'])
data = onehot_encoding(result,columns_to_onehot)

In [36]:
data.to_csv("business_one_hot_with_attribute.csv")

<font face="微软雅黑" size=6 color=#000000>predict

In [99]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from  sklearn.neural_network import MLPClassifier
#from xgboost import XGBClassifier
import os
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import random
import matplotlib.pyplot as plt
#from fastFM.mcmc import FMClassification
import sys
sys.setrecursionlimit(10000)
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing

In [100]:
x  = pd.read_csv("business_one_hot_with_attribute.csv")
#x = data
y = pd.read_csv('business.csv')['stars']

In [101]:
#x.fillna(0)
x = x.replace(np.nan, 0, regex=True)

# avoid inf value
#np.where(x.values >= np.finfo(np.float64).max)

In [102]:
x = x.drop('Unnamed: 0', axis=1)
x = x.drop('Unnamed: 0.1', axis=1)
#x = x.drop('RestaurantsPriceRange2', axis=1)

In [103]:
x.shape

(192609, 3882)

In [104]:
# x  = pd.read_csv("business_one_hot.csv")
# y = pd.read_csv('business.csv')['stars']
x_train =  x[:int(len(x)/5*4)]
x_validation =x[int(len(x)/5*4):]
y_train = y[:int(len(y)/5*4)]
y_validation = y[int(len(y)/5*4):]

In [105]:
def randomforest(x_train,y_train,Validation_x, n_estimators = 100, max_depth = 3,  min_samples_split = 6 , min_samples_leaf = 4, random_state = 500 ):
    rf = RandomForestClassifier(n_estimators=n_estimators,random_state = random_state)
    
#     lab_enc = preprocessing.LabelEncoder()
#     training_scores_encoded = lab_enc.fit_transform(y_train)
#     rf_model = rf.fit(x_train, training_scores_encoded)
    y_train = y_train*2
    rf_model = rf.fit(x_train, y_train)
    
    rf_p = rf_model.predict(Validation_x)
    rf_p = pd.DataFrame(rf_p)
    RF = rf_p/2
    return RF

In [17]:
RF = randomforest(x_train,y_train,x_validation, n_estimators = 100, max_depth = 5,  min_samples_split = 6, min_samples_leaf = 6, random_state = 500)

In [18]:
RF

,0
0,1.0
1,5.0
2,4.5
3,3.0
4,3.0
5,5.0
6,3.0
7,4.0
8,3.5
9,2.0


In [12]:
y_validation[1:10]

154088    5.0
154089    4.5
154090    3.5
154091    3.0
154092    5.0
154093    3.5
154094    4.0
154095    2.0
154096    3.5
Name: stars, dtype: float64

In [456]:
# n_estimators = 100
# max_depth = 5
# min_samples_split = 6
# min_samples_leaf = 6
# random_state = 500

In [19]:
RF.to_csv("RF.csv")

<font face="微软雅黑" size=6 color=#000000> Evaluation

In [32]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer

In [25]:
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(y_test)
    
    y_test = lb.transform(y_test)
    y_pred = lb.transform(y_pred)
    
    return roc_auc_score(y_test, y_pred, average=average)

In [36]:
# Accurancy
roc_auc = multiclass_roc_auc_score(y_validation*2, RF*2)

In [37]:
roc_auc

0.5618883269469791

<font face="微软雅黑" size=6 color=#000000> Prediction Example

In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier


x  = pd.read_csv("business_one_hot_with_attribute.csv")
y = pd.read_csv('business.csv')['stars']

x = x.replace(np.nan, 0, regex=True)

x = x.drop('Unnamed: 0', axis=1)
x = x.drop('Unnamed: 0.1', axis=1)

x_train =  x[:int(len(x)/5*4)]
x_validation =x[int(len(x)/5*4):]
y_train = y[:int(len(y)/5*4)]
y_validation = y[int(len(y)/5*4):]

def randomforest(x_train,y_train,Validation_x, n_estimators = 100, max_depth = 3,  min_samples_split = 6 , min_samples_leaf = 4, random_state = 500 ):
    rf = RandomForestClassifier(n_estimators=n_estimators,random_state = random_state)
    
#     lab_enc = preprocessing.LabelEncoder()
#     training_scores_encoded = lab_enc.fit_transform(y_train)
#     rf_model = rf.fit(x_train, training_scores_encoded)
    y_train = y_train*2
    rf_model = rf.fit(x_train, y_train)
    
    rf_p = rf_model.predict(Validation_x)
    rf_p = pd.DataFrame(rf_p)
    RF = rf_p/2
    return RF



/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
predict_target  = pd.DataFrame().reindex_like(pd.DataFrame(x.loc[1])).fillna(0).T

In [3]:
pd.set_option('display.max_columns', 1000)
predict_target

,categories_ & Probates,categories_ 3D Printing,categories_ ATV Rentals/Tours,categories_ Acai Bowls,categories_ Accessories,categories_ Accountants,categories_ Acne Treatment,categories_ Active Life,categories_ Acupuncture,categories_ Addiction Medicine,categories_ Adoption Services,categories_ Adult,categories_ Adult Education,categories_ Adult Entertainment,categories_ Advertising,categories_ Aerial Fitness,categories_ Aerial Tours,categories_ Aestheticians,categories_ Afghan,categories_ African,categories_ Air Duct Cleaning,categories_ Aircraft Dealers,categories_ Aircraft Repairs,categories_ Airlines,categories_ Airport Lounges,categories_ Airport Shuttles,categories_ Airport Terminals,categories_ Airports,categories_ Airsoft,categories_ Allergists,categories_ Alternative Medicine,categories_ Amateur Sports Teams,categories_ American (New),categories_ American (Traditional),categories_ Amusement Parks,categories_ Anesthesiologists,categories_ Animal Assisted Therapy,categories_ Animal Physical Therapy,categories_ Animal Shelters,categories_ Antiques,categories_ Apartment Agents,categories_ Apartments,categories_ Appliances,categories_ Appliances & Repair,categories_ Appraisal Services,categories_ Aquarium Services,categories_ Aquariums,categories_ Arabian,categories_ Arcades,categories_ Archery,categories_ Architects,categories_ Architectural Tours,categories_ Argentine,categories_ Armenian,categories_ Art Classes,categories_ Art Galleries,categories_ Art Museums,categories_ Art Restoration,categories_ Art Schools,categories_ Art Space Rentals,categories_ Art Supplies,categories_ Art Tours,categories_ Artificial Turf,categories_ Arts & Crafts,categories_ Arts & Entertainment,categories_ Asian Fusion,categories_ Assisted Living Facilities,categories_ Astrologers,categories_ Attraction Farms,categories_ Auction Houses,categories_ Audio/Visual Equipment Rental,categories_ Audiologist,categories_ Australian,categories_ Austrian,categories_ Auto Customization,categories_ Auto Detailing,categories_ Auto Glass Services,categories_ Auto Insurance,categories_ Auto Loan Providers,categories_ Auto Parts & Supplies,categories_ Auto Repair,categories_ Auto Security,categories_ Auto Upholstery,categories_ Automotive,categories_ Aviation Services,categories_ Awnings,categories_ Axe Throwing,categories_ Ayurveda,categories_ Baby Gear & Furniture,categories_ Backflow Services,categories_ Badminton,categories_ Bagels,categories_ Bail Bondsmen,categories_ Bakeries,categories_ Balloon Services,categories_ Bangladeshi,categories_ Bankruptcy Law,categories_ Banks & Credit Unions,categories_ Bar Crawl,categories_ Barbeque,categories_ Barbers,categories_ Barre Classes,categories_ Bars,categories_ Bartenders,categories_ Bartending Schools,categories_ Baseball Fields,categories_ Basketball Courts,categories_ Basque,categories_ Battery Stores,categories_ Batting Cages,categories_ Beach Bars,categories_ Beach Equipment Rentals,categories_ Beach Volleyball,categories_ Beaches,categories_ Beauty & Spas,categories_ Bed & Breakfast,categories_ Beer,categories_ Beer Bar,categories_ Beer Garden,categories_ Beer Gardens,categories_ Beer Hall,categories_ Beer Tours,categories_ Behavior Analysts,categories_ Belgian,categories_ Bespoke Clothing,categories_ Beverage Store,categories_ Bicycles,categories_ Bike Parking,categories_ Bike Rentals,categories_ Bike Repair,categories_ Bike Repair/Maintenance,categories_ Bike Sharing,categories_ Bike Shop,categories_ Bike tours,categories_ Bikes,categories_ Bingo Halls,categories_ Bird Shops,categories_ Bistros,categories_ Blood & Plasma Donation Centers,categories_ Blow Dry/Out Services,categories_ Boat Charters,categories_ Boat Dealers,categories_ Boat Parts & Supplies,categories_ Boat Repair,categories_ Boat Tours,categories_ Boating,categories_ Bocce Ball,categories_ Body Contouring,categories_ Body Shops,categories_ Bookbinding,categories_ Bookkeepers,categories_ Books,categories_ Bookstores,categories_ Boot Camps

In [4]:
# predict_target['NoiseLevel_quiet'] =1
# predict_target['Wheelchair_Accessible_True']  =1
# predict_target['WiFi_free'] = 1
# predict_target['OutdoorSeating_True'] =1
# predict_target['WheelchairAccessible_True'] = 1

# predict_target['ByAppointmentOnly_True'] =1
# predict_target['BusinessAcceptsCreditCards_True'] =1
# predict_target['BusinessParking.garage_True']=1 
# predict_target['GoodForKids_True'] =1


# predict_target['RestaurantsPriceRange2'] = 2
# predict_target['RestaurantsDelivery_True']
# predict_target['Alcohol_full_bar'] 

In [5]:
rf = RandomForestClassifier(n_estimators=100,random_state = 5)
    
y_train = y_train*2
rf_model = rf.fit(x_train, y_train)


In [11]:
############################################################
######### Restaurants #########################################
############################################################

# add your attribute here
# city/state/categories/attribute
predict_target  = pd.DataFrame().reindex_like(pd.DataFrame(x.loc[1])).fillna(0).T

#predict_target['city_phoenix'] = 1
predict_target['state_AZ'] = 1

predict_target['categories_Restaurants'] = 1

predict_target['Alcohol_full_bar']  = 1
predict_target['RestaurantsDelivery_True'] = 1
predict_target['NoiseLevel_quiet'] = 1
predict_target['OutdoorSeating_True'] = 1
predict_target['RestaurantsPriceRange2'] = 2
predict_target['WiFi_free'] = 1



In [12]:
rf_p = rf_model.predict(predict_target)
rf_p = pd.DataFrame(rf_p)
RF = rf_p/2

In [13]:
RF

,0
0,4.0


In [56]:
############################################################
######### Beauty and Spas #####################################
############################################################

# add your attribute here
# city/state/categories/attribute
predict_target  = pd.DataFrame().reindex_like(pd.DataFrame(x.loc[1])).fillna(0).T

#predict_target['city_phoenix'] = 1
predict_target['state_AZ'] = 1

predict_target['categories_Beauty & Spas'] = 1

predict_target['ByAppointmentOnly_True'] =1
predict_target['RestaurantsPriceRange2'] = 2 # only RestaurantsPriceRange2 avaliable
predict_target['GoodForKids_True']  = 1
predict_target['WheelchairAccessible_True'] = 1



In [57]:
rf_p = rf_model.predict(predict_target)
rf_p = pd.DataFrame(rf_p)
RF = rf_p/2

In [58]:
RF

,0
0,5.0


In [10]:
predict_target  = pd.DataFrame().reindex_like(pd.DataFrame(x.loc[1])).fillna(0).T

predict_target['state_AZ'] = 1

predict_target['categories_Restaurants'] = 1

rf_p = rf_model.predict(predict_target)
rf_p = pd.DataFrame(rf_p)
RF = rf_p/2
RF

,0
0,2.5


In [9]:
predict_target  = pd.DataFrame().reindex_like(pd.DataFrame(x.loc[1])).fillna(0).T

predict_target['state_AZ'] = 1

predict_target['categories_Beauty & Spas'] = 1

rf_p = rf_model.predict(predict_target)
rf_p = pd.DataFrame(rf_p)
RF = rf_p/2
RF

,0
0,3.5
